In [19]:
import random
import numpy as np
from numpy.linalg import inv

In [299]:
#### Helper functions ####

def generate_point():
    '''Generate a random point in [-1, 1] x [-1, 1]'''
    return [random.uniform(-1, 1), random.uniform(-1, 1)]

def define_f():
    '''Return a slope and y-intercept based on two random points in
       [-1, 1] x [-1, 1]. This defines the function f(x) = slope * x + y_int. 
       This can fail if f is vertical. However, this is extremely unlikely given 
       that our two points are chosen randomly. If it does fail, we'll get an error.'''
    p1 = generate_point()
    p2 = generate_point()
    slope = (p2[1] - p1[1]) / (p2[0] - p1[0])
    y_int = (-1 * slope * p1[0]) + p1[1] # From point-slope form
    return slope, y_int

def evaluate_point(pt, slope, y_int):
    ''' Return 1 if the point is above f, -1 otherwise'''
    if pt[1] >= slope * pt[0] + y_int: # pt[0] is x coord, pt[1] is y coord
        return 1
    return -1

def build_training_set(N, slope, y_int):
    '''Build a set of N training points of the form ([1.0, x1, x2], y). The 1.0 is the
       artificial coordinate used to simplify the math. x1 and x2 come from
       [-1, 1] x [-1, 1], and y is -1 or +1, depending on whether the point (x1, x2) is
       above or below f. f is defined by f(x) = slope * x + y_int'''
    training_set = []
    for i in range(N):
        xn = generate_point() 
        yn = evaluate_point(xn, slope, y_int)
        training_set.append(([1.] + xn, yn)) # Note the artificial coordinate
    return training_set
    
def eval_PLA(pt, weights):
    '''Classify the given point as +1 or -1 based on the the given weights vector'''
    return np.sign(np.dot(weights, np.array(pt)))

def train_PLA(training_set, W=None):
    '''Train the PLA on training_set until it converges to f. 
       Return the number of iterations needed, along with the 
       final weight vector.'''
    if W is None:
        W = np.zeros(3) # weight vector; we use 3 dimensions because we have x, y, 
                        # and the threshold
    num_iters = 0
    missed_pts = [pair for pair in training_set if pair[1] != eval_PLA(pair[0], W)]
    
    while missed_pts != []:
        missed_pt = random.choice(missed_pts)
        missed_y = missed_pt[1]
        missed_x = np.array(missed_pt[0]) # this is a vector
        W += missed_y * missed_x # Update the weight vector
        missed_pts = [pair for pair in training_set if pair[1] != eval_PLA(pair[0], W)]
        num_iters += 1
        
    return num_iters, W

def compute_linreg_weights(X_mat, y_vec):
    '''Takes a matrix of training values X_mat (each row is a training instance) and
       a target vector y_vec and computes the linear regression weight vector for the
       data.'''
    pseudo_inv = np.dot(inv((np.dot(X_mat.T, X_mat))), X_mat.T)
    return np.dot(pseudo_inv, y_vec)

def predict_linreg(x, weights):
    '''Uses a pre-trained linear regression model (given by weights) to predict a binary
       target function given data x.'''
    print(weights)
    print(x)
    return np.sign(np.dot(weights, x))


# Problem 5

In [199]:
lines = []
weights_vectors = []
in_sample_errors = []
N = 100

for _ in range(1000):
    slope, y_int = define_f()
    lines.append((slope, y_int))
    training_set = build_training_set(N, slope, y_int)

    # Do our linear regression
    X = np.array([ent[0] for ent in training_set])
    Y = np.array([ent[1] for ent in training_set], dtype=float)
    weights = compute_linreg_weights(X, Y)
    weights_vectors.append(weights)

    # Find the in-sample error
    predictions = np.array([predict_linreg(np.array(ent[0]), weights) for ent in training_set])
    # Make a vector whose entries are 1 if the prediction and ground truth agree, 0 otherwise.
    # Then the percent correct is the number of 1's in this vector divided by its length,
    # and the percent error is 1 - percent correct
    in_sample_error = 1 - sum(np.equal(Y, predictions).astype(int)) / N
    in_sample_errors.append(in_sample_error)

In [200]:
sum(in_sample_errors) / len(in_sample_errors)

0.038920000000000121

This is closest to answer [c]: 0.01.

# Problem 6

In [75]:
oos_errors = []
TEST_SET_SIZE = 1000
# Iterate through the 1000 saved (f, weights) configs from Problem 5
for i in range(1000):
    slope, y_int = lines[i]
    weights = weights_vectors[i]
    
    # Generate 1000 points
    testing_set = build_training_set(TEST_SET_SIZE, slope, y_int) # not really a "training set"
    
    # Compute the error on this testing set (out of sample error)
    ground_truths = np.array([ent[1] for ent in testing_set], dtype=float)
    predictions = np.array([predict_linreg(np.array(ent[0]), weights) for ent in testing_set])
    oos_error = 1 - sum(np.equal(ground_truths, predictions).astype(int)) / TEST_SET_SIZE
    oos_errors.append(oos_error)


In [76]:
sum(oos_errors) / len(oos_errors)

0.047604999999999911

This is closest to answer [c]: 0.01.

# Problem 7

In [148]:
sum_num_iters = 0
for _ in range(1000):
    slope, y_int = define_f()
    training_set = build_training_set(10, slope, y_int)

    # Do our linear regression
    X = np.array([ent[0] for ent in training_set])
    Y = np.array([ent[1] for ent in training_set], dtype=float)
    weights = compute_linreg_weights(X, Y)

    # Pass these weights to PLA
    num_iters, _ = train_PLA(training_set, W=weights)
    sum_num_iters += num_iters

In [149]:
sum_num_iters / 1000

3.566

This is closest to answer [a]: 1.

# Problem 8

In [193]:
import copy

def f(pt):
    '''Implements the function f(x1, x2) = sign(x1^2 + x2^2 - 0.6),
       where x1 and x2 are interpreted as the two components'''
    return np.sign(pt[0]**2 + pt[1]**2 - 0.6)

def build_dataset(N, target_func):
    '''Build a data set of N points and use target_func to evaluate each
       point to get the ground truth values.'''
    dataset = []
    for _ in range(N):
        x = generate_point()
        y = target_func(x)
        dataset.append(([1.] + x, y))
    return dataset

def noisify_dataset(dataset, ratio):
    '''Randomly flip the sign of the output in ratio of a given dataset. A new noisy dataset
       is returned, rather than modifying the original inplace.'''
    dataset = copy.deepcopy(dataset)
    num_elems = len(dataset)
    idx_subset = np.random.choice(range(num_elems), size=int(ratio * num_elems), replace=False)
    for i in idx_subset:
        val = dataset[i][1]
        flipped_val = 1.0 if val == -1.0 else -1.0
        dataset[i] = (dataset[i][0], flipped_val)
    return dataset
    

In [204]:
# Now we can do our simulation
in_sample_error_sum = 0
NUM_TRIALS = 1000
TRAINING_SIZE = 1000

for _ in range(NUM_TRIALS):
    dataset = build_dataset(TRAINING_SIZE, f)
    dataset = noisify_dataset(dataset, .1) # Randomly flip 10% of the dataset output values

    # Do our linear regression
    X = np.array([ent[0] for ent in dataset])
    Y = np.array([ent[1] for ent in dataset], dtype=float)
    weights = compute_linreg_weights(X, Y)

    # Find the in-sample error
    predictions = np.array([predict_linreg(np.array(ent[0]), weights) for ent in dataset])
    in_sample_error = 1 - sum(np.equal(Y, predictions).astype(int)) / TRAINING_SIZE
    in_sample_error_sum += in_sample_error

In [205]:
in_sample_error_sum / NUM_TRIALS

0.50546599999999964

This is closest to [d] 0.5. This makes sense because the data are highly linearly inseparable; thus, the performance is very poor.

# Problem 9

In [206]:
def nonlinear_transform(x):
    '''Converts a data point of the form [1 x1 x2] to the form
       [1, x1, x2, x1x2, x1^2 x2^2].'''
    return [1.0, x[1], x[2], x[1] * x[2], x[1] * x[1], x[2] * x[2]]

In [264]:
def eval_a(x):
    return np.sign(-1 - 0.05*x[1] + 0.08*x[2] + .13*x[1]*x[2] + 1.5*x[1]**2 + 1.5*x[2]**2)

def eval_b(x):
    return np.sign(-1 - 0.05*x[1] + 0.08*x[2] + .13*x[1]*x[2] + 1.5*x[1]**2 + 15*x[2]**2)

def eval_c(x):
    return np.sign(-1 - 0.05*x[1] + 0.08*x[2] + 0.13*x[1]*x[2] + 15*x[1]**2 + 1.5*x[2]**2)

def eval_d(x):
    return np.sign(-1 - 1.5*x[1] + .08*x[2] + .13*x[1]*x[2] + 0.05*x[1]**2 + .05*x[2]**2)

def eval_e(x):
    return np.sign(-1 - .05*x[1] + 0.08*x[2] + 1.5*x[1]*x[2] + 0.15*x[1]**2 + 0.15*x[2]**2)

In [283]:
dataset = build_dataset(1000, f)
dataset = noisify_dataset(dataset, .1) # Randomly flip 10% of the dataset output values

# Nonlinearly transform the dataset
X = np.array([ent[0] for ent in dataset])
X_transform = np.array([nonlinear_transform(x) for x in X])
Y = np.array([ent[1] for ent in dataset], dtype=float)
# Train on the transformed data
weights = compute_linreg_weights(X_transform, Y)

# Compare the trained model vs all of the multiple choice options
a_vs_trained = [float(predict_linreg(x, weights) == eval_a(x)) for x in X_transform]
b_vs_trained = [float(predict_linreg(x, weights) == eval_b(x)) for x in X_transform]
c_vs_trained = [float(predict_linreg(x, weights) == eval_c(x)) for x in X_transform]
d_vs_trained = [float(predict_linreg(x, weights) == eval_d(x)) for x in X_transform]
e_vs_trained = [float(predict_linreg(x, weights) == eval_e(x)) for x in X_transform]

a_error = 1 - sum(a_vs_trained) / 1000
b_error = 1 - sum(b_vs_trained) / 1000
c_error = 1 - sum(c_vs_trained) / 1000
d_error = 1 - sum(d_vs_trained) / 1000
e_error = 1 - sum(e_vs_trained) / 1000

In [284]:
print(a_error)
print(b_error)
print(c_error)
print(d_error)
print(e_error)

0.015000000000000013
0.34099999999999997
0.365
0.347
0.43200000000000005


The lowest error is [a], so that is our answer.

# Problem 10

In [297]:
TEST_SET_SIZE = 1000
NUM_TRIALS = 1000

oos_error_sum = 0
for _ in range(NUM_TRIALS):
    test_set = build_dataset(TEST_SET_SIZE, f)
    test_set = noisify_dataset(test_set, 0.1)
    
    # Nonlinearly transform the dataset
    X = np.array([ent[0] for ent in test_set])
    X_transform = np.array([nonlinear_transform(x) for x in X])
    Y = np.array([ent[1] for ent in test_set], dtype=float)

    predictions = np.array([predict_linreg(x, weights) for x in X_transform])
    oos_error = 1 - sum(np.equal(Y, predictions).astype(int)) / TEST_SET_SIZE
    oos_error_sum += oos_error

In [298]:
oos_error_sum / 1000

0.13973900000000034

This is closest to [b]: 0.1, so that is our answer.